# SEE110 Klimatmodellering: Glaciärer i Skandinavien under 2000-talet
Introduktions-notebook för projektet Glaciärmodellering i kursen SEE110 Klimatmodellering.

<div>
<div>
<img src="https://docs.oggm.org/en/stable/_static/logos/oggm_l_alpha.png" width="300" align="left"/>
</div>
<div>
<img src="https://edu.oggm.org/en/latest/_static/logos/oggm_edu_bulb_l_alpha.png" width="300" align="left"/>
</div>
</div>

In [ ]:
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import oggm.cfg
import pandas as pd
import salem
import seaborn as sns
import see110_utils
import xarray as xr
from oggm import graphics, tasks, utils, workflow

sns.set_context("notebook")  # plot defaults

In [ ]:
# OGGM options
oggm.cfg.initialize(logging_level="WARNING")
oggm.cfg.PARAMS["min_ice_thick_for_length"] = 1  # a glacier is when ice thicker than 1m
oggm.cfg.PARAMS["store_model_geometry"] = True

## Bestämma projektmapp
Det första du måste göra är att bestämma vart du vill att projektet och dess data ska sparas.
Detta gör du genom att sätta `oggm.cfg.PATHS["working_dir"]` till den absoluta sökvägen för mappen.



In [ ]:
# Detta är en temporär mapp. Använde inte denna för att arbeta med ditt projekt.
oggm.cfg.PATHS["working_dir"] = utils.gettempdir(dirname="see110-project")
# Exempel på absolut sökväg.
# oggm.cfg.PATHS["working_dir"] = "/home/erik/data/see110-project

## Definiera vilka glaciärer vi ska simulera
Här laddar vi ner en lista med id-nummer ([Randolph Glacier Inventory](https://nsidc.org/data/nsidc-0770/versions/6), RGI) för så kallade referensglaciärer.
Vi använder [Pandas](https://pandas.pydata.org/) för att läsa in listan.

In [ ]:
rgi_df = pd.read_csv(
    "https://raw.githubusercontent.com/OGGM/oggm-sample-data/master/wgms/rgi_wgms_links_20220112.csv",
    header=0,
)

Vi kan välja ut glaciärerna i Sverige såhär:

In [ ]:
rgi_df[rgi_df.POLITICAL_UNIT == "SE"]

Den här listan innehåller massor av information, för att köra modellen är det kolumnen `RGI60_ID` vi är intresserade av.
Vi kan läsa id-numret för Storglaciären (för övrigt en av glaciärerna med längst observationsserie i världen) och skapa en variabel för detta.

In [ ]:
# Storglaciären
rgi_id = "RGI60-08.00213"

###  Överkurs: Arbeta direkt med RGI
<div class="alert alert-info">
   <b>Info: </b>Den här delen kan du hoppa över första gången du går igenom den här notebooken, och helt enkelt scrolla ner till "Ladda ner glaciärdata".
    Men den kan vara intressant att komma tillbaka till när ni kommit igång med era projekt.
</div>
    
Filen ovan innehåller ett urval av alla glaciärer i världen, men det finns fler än 200 000 stycken!
Om ni under ert projekts gång är intresserade av att simulera fler, eller andra, glaciärer än de som finns med i listan ovan behöver ni arbeta med Randolph Glacier Inventory (RGI) direkt.

RGI är en stor datamängd som kräver lite kunskap för att hantera på ett smidigt sätt.
Det första vi måste göra är att ladda ner all RGI-data.
Det gör vi med följande funktion:

In [ ]:
utils.get_rgi_dir(version="62")

RGI-databasen är indelad i regionerna som visas i kartan nedan.
Det första vi måste göra för att hitta glaciärerna vi är intresserade av är att välja region.


<img alt="Världskarta som visar RGI-regionerna" src="https://raw.githubusercontent.com/GLIMS-RGI/rgi_user_guide/main/docs/img/global_stats/global_map.jpeg" style="width:1100px" />

|01 Region|Full name|
|--------|---------|
|01|Alaska|
|02|Western Canada and USA|
|03|Arctic Canada North|
|04|Arctic Canada South|
|05|Greenland Periphery|
|06|Iceland|
|07|Svalbard and Jan Mayen|
|08|Scandinavia|
|09|Russian Arctic|
|10|North Asia|
|11|Central Europe|
|12|Caucasus and Middle East|
|13|Central Asia|
|14|South Asia West|
|15|South Asia East|
|16|Low Latitudes|
|17|Southern Andes|
|18|New Zealand|
|19|Subantarctic and Antarctic Islands|
|20|Antarctic Mainland|


För mer info, se: https://www.glims.org/rgi_user_guide/appendix/regions.html#o1-regions-table

Här väljer vi ut region 08, Skandinavien:

In [ ]:
file_path = utils.get_rgi_region_file("08")

Filen som sökvägen ovan pekar på är en så kallad shapefil, som lagrar geografisk data.

Vi kan öppna den med `GeoPandas`, här förkortat som `gpd`:

In [ ]:
gdf = gpd.read_file(file_path)

Här ser vi att första kolumnen innehåller `RGIID`, medan andra kolumner innehåller t.ex. area, meidanhöjd (Zmed), och lutning (Slope).

In [ ]:
gdf.head()

Vi kan också kolla hur många glaciärer det finns i regionen:

In [ ]:
len(gdf)

Detta är lite för många glaciärer att simulera för det här projektet, med tanke på vilka resurser ni har, så ni behöver fundera på hur ni kan välja ut en grupp glaciärer som passar in i era frågeställningar. 

Nedan följer ett exempel där vi väljer ut alla glaciärer som är större än 2 km<sup>2</sup>, och har ett namn.

In [ ]:
gdf_selection = gdf[gdf.Area > 2]

In [ ]:
gdf_selection = gdf_selection[~gdf_selection.Name.isnull()]

Hur många glaciärer är kvar?

In [ ]:
len(gdf_selection)

Vi kan dela upp detta ytterligare genom att titta på under-regionerna i Skandinavien.
Tar vi en titt på [följande sida](https://www.glims.org/rgi_user_guide/appendix/regions.html#second-order-regions), ser vi att Skandinavien har följande under-regioner:
- 1 : North Scandinavia
- 2 : Southwest Scandinavia
- 3 : Southeast Scandinavia

Vi väljer ut alla glaciärer i Southwest Scandinavia såhär:

In [ ]:
n_scand = gdf_selection[gdf_selection['O2Region'] == "1"]

Vi tar sedan fram deras respektive RGI_ID såhär:

In [ ]:
rgi_ids = n_scand.RGIId

Vilka vi kan använda för att köra våra simuleringar.

### Ladda ner glaciärdata

Nästa steg är att ladda ner den data som behövs för att köra modellen för glaciären vi har valt ut.
Här behöver vi ange några parametrar som kontrollerar vilken typ av input-data vi kommer använda, men detta är inget du behöver förstå nu.
Notera dock att vi skickar med vår `rgi_id` variabel i en lista.
Mer om detta senare.

Detta kan ta några minuter.

In [ ]:
# We pick the elevation-bands glaciers because they run a bit faster - but they create more step changes in the area outputs
base_url = "https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.3/elev_bands/W5E5_spinup"

In [ ]:
# Download gdir for a single rgi_id.
gdir = workflow.init_glacier_directories(
    [rgi_id], from_prepro_level=5, prepro_border=160, prepro_base_url=base_url
)[0]


Cellen ovan returnerar något som kallas för en `GlacierDirectory` - vilket förkortas `gdir` (och vårt variabelnamn).
En `gdir` innehåller all information om en glaciär som modellen behöver.

### Interaktiv karta
Med `see110_utils.plot_glacier_map` kan vi visa en interaktiv karta över glaciären i `gdir`:

In [ ]:
out = see110_utils.plot_glacier_map(gdir)
out

## Historiska simuleringar
Input-datan vi använder inkluderar något som kallas för en "spinup".
Detta är en simulering av glaciären som försöker återskapa hur glaciären kan ha sett ut historiskt.
Då det för de flesta av världens glaciärer finns inga, eller väldigt få, observationer är detta svårt, och utanför vad vi behöver kunna för den här kursen.
Men kort innefattar det att iterativt hitta en kombination av en historisk glaciärarea och modellparametrar som leder till att en historisk simulering resulterar i att arean för den simulerade glaciären matchar de tidigaste observationerna.

Nedan öppnar vi filen med spinup-datan och plottar volymen och arean från 1979.

In [ ]:
with xr.open_dataset(
    gdir.get_filepath("model_diagnostics", filesuffix="_spinup_historical")
) as ds:
    ds = ds.sel(time=slice(1980, 2020)).load()

In [ ]:
fig, axs = plt.subplots(nrows=2, figsize=(10, 7), sharex=True)
ds.volume_m3.plot(ax=axs[0])
ds.area_m2.plot(ax=axs[1])
axs[1].scatter(
    gdir.rgi_date + 1, gdir.rgi_area_m2, label=f"Area {gdir.rgi_date+1}", c="C1"
)
axs[0].set_xlabel("")
axs[0].set_title(f"{rgi_id}")
axs[1].set_xlabel("Years")
plt.legend();

Här kan vi att arean för den historiska simuleringen stämmer relativt väl med obsevationen från 2003.

## Simulera glaciärer under 2000-talet

Nu är det dags att göra några simuleringar för de kommande ~80 åren.
Först ska vi välja vilka glaciärer vi vill simulera och förbereda input-datan (gdirs) för dessa.

Här väljer vi ut alla referensglaciärer i Sverige:

In [ ]:
rgi_ids = rgi_df[rgi_df.POLITICAL_UNIT == "SE"].RGI60_ID.values
print(rgi_ids)

och laddar ner inputdatan:

In [ ]:
gdirs = workflow.init_glacier_directories(
    rgi_ids, from_prepro_level=5, prepro_border=160, prepro_base_url=base_url
)

Notera att `gdirs` nu är en lista med data för sju glaciärer.

In [ ]:
# Ta bort kommentaren för att se vad gdirs innehåller.
# gdirs

Vi kan visa en karta för någon av glaciärerna i listan:

In [ ]:
out = see110_utils.plot_glacier_map(gdirs[0])
out

### Data för klimatprojektioner
För att modellera glaciärerna fram till 2100 måste vi först ladda ner och förbereda klimatprojektioner från en global klimatmodell (eller flera). 
Här kan du välja en av följande:

|Modell|
|----|
|gfdl-esm4_r1i1p1f1 |
|mpi-esm1-2-hr_r1i1p1f1 |
|mri-esm2-0_r1i1p1f1 |
|ipsl-cm6a-lr_r1i1p1f1 |
|ukesm1-0-ll_r1i1p1f2 |

och vi definiera variabeln `member`:

In [ ]:
member = "mpi-esm1-2-hr_r1i1p1f1"

Du behöver också välja vilket SSP-scenario du vill använda (ssp126, ssp370, ssp585).
Liknande innan, definierar vi variabeln `ssps`, som kommer användas i följande celler.
Notera att detta är en lista som kan hålla ett eller flera ssp scenarion.

In [ ]:
ssps = ["ssp126"]

Cellen nedan laddar ner och förbehandlar datan.

In [ ]:
from oggm.shop import gcm_climate

# Download the three main SSPs
for ssp in ssps:
    # bias correct them
    workflow.execute_entity_task(
        gcm_climate.process_monthly_isimip_data,
        gdirs,
        ssp=ssp,
        # gcm member -> you can choose another one
        member=member,
        # recognize the climate file for later
        output_filesuffix=f"_ISIMIP3b_{member}_{ssp}",
    );

### Simulera glaciärerna

Nu är du redo att starta simuleringarna.

In [ ]:
# Först loopar vi över ssp scenarion.
for ssp in ssps:
    # Här definierar vi namnet på datan som vi vill använda.
    rid = f"_ISIMIP3b_{member}_{ssp}"
    # Funktion som kör modellen.
    workflow.execute_entity_task(
        # Vi specifierar att vi vill köra modellen med hydrologiska outputs påslagna.
        tasks.run_with_hydro,
        # Vilka gdirs ska modellin köras på.
        gdirs,
        # Modellen ska köras från klimatdata.
        run_task=tasks.run_from_climate_data,
        # Specifiera vilket år körningen ska börja.
        ys=2020,
        # use gcm_data, not climate_historical
        climate_filename="gcm_data",
        # use the chosen scenario
        climate_input_filesuffix=rid,
        # this is important! Start from 2020 glacier
        init_model_filesuffix="_spinup_historical",
        # recognize the run for later
        output_filesuffix=rid,
        # add monthly diagnostics
        store_monthly_hydro=True,
    );

### En snabbtitt på datan
Körningen ovan genererar en fil per glaciär och modell och SSP-scenario.
För att enklare analysera datan kan vi slå samman dessa med hjälp av `utils.compile_run_output`.

Detta kommer även spara den sammanslagna datan som en netcdf fil i din `WORKING_DIR`.

In [ ]:
# Här kombinerar vi de separata filerna till en.
ds = utils.compile_run_output(gdirs, input_filesuffix=rid)

Variabeln `ds` är här ett så kallat dataset från biblioteket [xarray](https://xarray.dev/).
Ett dataset representerar en, eller flera netcdf-filer, och gör det relativt enkelt att analysera och visualisera den underliggande datan.
Kör vi en cell med endast variabeln `ds` kan vi se vad det innehåller.

In [ ]:
ds

Nedan plottar vi volymen för glaciärerna vi simulerat från 2020 till 2100.
Vi inkluderar också den totala volymen för alla glaciärerna genom summera över id-numret `.sum("rgi_id")`.

In [ ]:
# Först loopar vi över de individuella glaciärerna i datasetet.
#for rgi_id in ds.rgi_id.values:
#    sel = ds.volume.sel(rgi_id=rgi_id)
#    sel.plot(label=sel.rgi_id.values, c="C0")
#
ds.volume.sum("rgi_id").plot(label="Median")
#plt.legend()
plt.title(f"Glaciärvolym i Sverige för\n{member}, {ssp}");

## Fortsatt arbete med ditt projekt
Nu är det dags att fortsätta med analysen för ditt projekt.
Du kan helt enkelt fortsätta med att lägga till nya celler här under, eller så skapar du en ny notebook.
Kom i så fall ihåg att kopiera över cellerna som importerar de bibliotek du behöver och konfigurerar OGGM.

Lycka till!